In [29]:
import pandas as pd
import numpy as np
from itertools import combinations

def find_optimal_mills(file_path):
    # Load data and preprocess
    df = pd.read_csv(file_path)
    df['Min_Cost'] = df[['Trucking cost', 'Rail cost']].min(axis=1)
    
    # Create dictionary: {Mill_County: {County: Min_Cost}}
    mill_coverage = df.groupby('Mill County').apply(
        lambda x: dict(zip(x['County'], x['Min_Cost']))
    ).to_dict()
    
    all_counties = df['County'].unique()
    best_combo = None
    best_total = float('inf')
    best_breakdown = {}
    best_assignment = {}

    # Evaluate all 3-mill combinations
    for combo in combinations(mill_coverage.keys(), 3):
        total = 0
        valid = True
        
        for county in all_counties:
            costs = []
            for mill in combo:
                if county in mill_coverage[mill]:
                    costs.append(mill_coverage[mill][county])
            if not costs:
                valid = False
                break
            total += min(costs)
            
        if valid and total < best_total:
            best_total = total
            best_combo = combo
            # Calculate the cost breakdown and assignment for this combo
            current_breakdown = {}
            current_assignment = {mill: [] for mill in combo}
            for county in all_counties:
                min_cost = float('inf')
                selected_mill = None
                for mill in combo:
                    cost = mill_coverage[mill].get(county, float('inf'))
                    if cost < min_cost:
                        min_cost = cost
                        selected_mill = mill
                current_breakdown[county] = min_cost
                current_assignment[selected_mill].append(county)
            best_breakdown = current_breakdown
            best_assignment = current_assignment

    return best_combo, best_total, best_breakdown, best_assignment

# Example usage
input_file = "A_data.csv"  # Replace with your file path
mills, total_cost, cost_breakdown, mill_assignment = find_optimal_mills(input_file)
    
if mills:
    print("Optimal Mill Counties:", mills)
    print(f"Total Minimum Cost: ${total_cost:,.2f}")
    print("\nCost Breakdown by County:")
    for county, cost in cost_breakdown.items():
        print(f"- {county}: ${cost:.2f}")
    print("\nCounties Targeted by Each Mill:")
    for mill, counties in mill_assignment.items():
        print(f"{mill}:")
        print(", ".join(counties))
else:
    print("No valid combination found that covers all counties with 3 mills.")

Optimal Mill Counties: ('Ashley County', 'Lincoln County', 'Putnam County')
Total Minimum Cost: $7,937.40

Cost Breakdown by County:
-  Cook County: $330.00
-  Broward County: $0.00
-  King County: $180.00
-  Dallas County: $195.00
-  Miami-Dade County: $0.00
-  Tarrant County: $195.00
-  Harris County: $195.00
-  Bexar County: $195.00
-  Alameda County: $538.20
-  Santa Clara County: $538.20
-  Kings County: $540.00
-  New York County: $540.00
-  Queens County: $540.00
-  Los Angeles County: $538.20
-  Clark County: $540.00
-  San Bernardino County: $538.20
-  Orange County: $538.20
-  Riverside County: $538.20
-  San Diego County: $538.20
-  Maricopa County: $720.00

Counties Targeted by Each Mill:
Ashley County:
 Cook County,  Dallas County,  Tarrant County,  Harris County,  Bexar County,  Maricopa County
Lincoln County:
 King County,  Alameda County,  Santa Clara County,  Los Angeles County,  Clark County,  San Bernardino County,  Orange County,  Riverside County,  San Diego County

/var/folders/8w/f1nm4bk161nf0bdc91693vr40000gn/T/ipykernel_20388/1270379262.py:11: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  mill_coverage = df.groupby('Mill County').apply(
